## **Configuración de entorno**

In [ ]:
import matplotlib
from matplotlib.pyplot import figure

In [ ]:
root = '/content/temp_drive'

In [ ]:
from google.colab import drive
drive.mount(root, force_remount=True)

In [ ]:
%cd "{root}/MyDrive/monografia_UDEA/Codigo"

In [ ]:
%run 00_configuracion_entorno.ipynb

**Análisis de la miníma fecha de cada móneda**

Se realiza con el objetivo de poder tomar una fecha de corte simmilar para todas las monedas y facilitar el analisis en la misma ventana de tiempo

In [ ]:
print("Miníma Fecha del bitcoin", bitcoin.Date.min())
print("Miníma Fecha del binance", binance.Date.min())
print("Miníma Fecha del cardano", cardano.Date.min())
print("Miníma Fecha del ethereum", ethereum.Date.min())
print("Miníma Fecha del tether", tether.Date.min())

In [ ]:
bitcoin.head()

In [ ]:
bitcoin.Date.max()

Se crea un dataframe vació con las columnas necesarias

In [ ]:
df_Coins_open = df = pd.DataFrame(columns=['bitcoinOpen', 'binanceOpen', 'cardanoOpen','ethereumOpen','tetherOpen'])

Se inicializan los campos del nuevo dataframe con los valores correspondientes

In [ ]:
df_Coins_open['bitcoinOpen'] = bitcoin.Open[bitcoin.Date >= '2017-10-02 23:59:59'].values
df_Coins_open['binanceOpen'] = binance.Open[binance.Date >= '2017-10-02 23:59:59'].values
df_Coins_open['cardanoOpen'] = cardano.Open[cardano.Date >= '2017-10-02 23:59:59'].values
df_Coins_open['ethereumOpen'] = ethereum.Open[ethereum.Date >= '2017-10-02 23:59:59'].values
df_Coins_open['tetherOpen'] = tether.Open[tether.Date >= '2017-10-02 23:59:59'].values

#Se agrega la fecha 
df_Coins_open['Date'] = pd.to_datetime(cardano['Date']).values

Se formatean los datos como tipo date y se indexa la fecha

In [ ]:
df_Coins_open['Date'] = df_Coins_open['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_Coins_open.set_index("Date", inplace=True)

In [ ]:
df_Coins_open.head()

Se toma la información relevante del dataframe que contiene los precios de apertura de cada moneda

In [ ]:
df_Coins_open.info()

## **Visualización Series de Tiempo monedas**

Se apunta a la ruta de salidas para almacenar las imagenes resultantes

In [ ]:
%cd "{root}/MyDrive/monografia_UDEA/Salidas"

**Bitcoin**

In [ ]:
matplotlib.rcParams.update({'font.size': 20, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})

In [ ]:
bitcoin.head(10)

In [ ]:
bitcoin.describe()

# Análisis de tendencia en escala logaritmica

## Grafica superpuesta de tendencia de las diferentes mónedas 

In [ ]:
figure(figsize=(20, 10), dpi=80)
bitcoinplt, = plt.plot(df_Coins_open.index,df_Coins_open.bitcoinOpen,label="Bitcoin")
binanceOpen, = plt.plot(df_Coins_open.index,df_Coins_open.binanceOpen,label="Binance")
cardanoOpen, = plt.plot(df_Coins_open.index,df_Coins_open.cardanoOpen,label="Cardano")
ethereumOpen, = plt.plot(df_Coins_open.index,df_Coins_open.ethereumOpen,label="Ethereum")
tetherOpen, = plt.plot(df_Coins_open.index,df_Coins_open.tetherOpen,label="Tether")
plt.title("Valor Apertura Criptomonedas", fontsize=15)
plt.xlabel("", fontsize=15)
plt.ylabel("Valor de Apertura (USD)", fontsize=15) 
plt.xticks(range(1,2000,400), ['2017','2018','2019','2020','2021'], fontsize=15)
plt.yticks(fontsize=15)
plt.legend(handles=[bitcoinplt, binanceOpen, cardanoOpen,ethereumOpen,tetherOpen], fontsize=12)
plt.savefig("time-series-tendency-Log-AllCoins.jpg", bbox_inches='tight')
plt.show()


## Datos desde 2018 hasta 2021
Consolidación de datos desde 2018-2021 y conversión a escala logaritmica

In [ ]:
df_Coins_open.head()

In [ ]:
df_Coins_open_copy = df_Coins_open.copy()
delete_df_idx=df_Coins_open_copy[df_Coins_open_copy.index<'2018-01-01'].index
df_Coins_open_copy=df_Coins_open_copy.drop(delete_df_idx)

In [ ]:
for key in df_Coins_open_copy.columns:
  df_Coins_open_copy[key] = df_Coins_open_copy[key].apply(lambda x: np.log10(x))

In [ ]:
df_Coins_open_copy.head()

## Visualización tendencia 

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(20,25), sharex=True)
fig.subplots_adjust(hspace=0)

ax[0].plot(df_Coins_open_copy['binanceOpen'], lw=3, label="Binance") 
ax[0].legend(fontsize=20)
ax[0].tick_params(labelsize=25)

ax[1].plot(df_Coins_open_copy['bitcoinOpen'], label="Bitcoin") 
ax[1].legend(fontsize=20)
ax[1].tick_params(labelsize=25)

ax[2].plot(df_Coins_open_copy['cardanoOpen'], label="Cardano") 
ax[2].legend(fontsize=20)
ax[2].tick_params(labelsize=25)

ax[3].plot(df_Coins_open_copy['ethereumOpen'], label="Ethereum") 
ax[3].legend(fontsize=20)
ax[3].tick_params(labelsize=25)

ax[4].plot(df_Coins_open_copy['tetherOpen'], label="Tether") 
ax[4].legend(fontsize=20)
ax[4].tick_params(labelsize=25)

fig.text(0.5, 0.04, 'Años', ha='center',fontsize=40)
fig.text(0.04, 0.5, 'Valor de Apertura (USD)', va='center', rotation='vertical', fontsize=40)

plt.xticks(range(1,1600,400), ['2018','2019','2020','2021'], fontsize=20)

plt.savefig("time-series-tendency.jpg", bbox_inches='tight')
plt.show()
